Instalar Bibliotecas pyspark e findspark

In [109]:
!pip install pyspark
!pip install findspark

Import bibliotecas

In [110]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [111]:
df = spark.sql('''select 'OK' as Status''')
df.show()

+------+
|Status|
+------+
|    OK|
+------+



## Manipulação de dados usando spark

In [112]:
from pyspark.sql import Row, DataFrame
from pyspark.sql.types import StringType, StructType, StructField, IntegerType, FloatType
from pyspark.sql.functions import col, expr, lit, substring, concat, concat_ws, when, coalesce
from pyspark.sql import functions as F
import pyspark.sql.types as T
from functools import reduce
from typing import List

In [113]:
df = spark.read.csv('/content/drive/MyDrive/PosTech_Analise_de_dados/FASE3/tech_challenge_3/dados/df_covid.csv', sep = ',', inferSchema=True, header=True)

print('df.count  :', df.count())
print('df.col ct :', len(df.columns))
print('df.columns:', df.columns)

df.count  : 1149197
df.col ct : 52
df.columns: ['uf', 'idade', 'sexo', 'cor', 'escolaridade', 'ensino_publico_privado', 'frequentou_escola', 'tipo_moradia', 'situacao_domiciliar', 'rendimento', 'restringiu_contato_fisico', 'febre', 'tosse', 'dor_garganta', 'dificuldade_respirar', 'dor_cabeca', 'dor_peito', 'nausea', 'nariz_entupido', 'fadiga', 'dor_olhos', 'perda_olfato_paladar', 'dor_muscular', 'diarreia', 'ida_unidade_saude', 'hospital_publico', 'hospital_privado', 'posto_publico', 'ambulatorio_privado', 'pronto_socorro_publico', 'pronto_socorro_privado', 'fez_teste', 'teste_swab', 'resultado_swab', 'teste_sangue_dedo', 'resultado_sangue_dedo', 'teste_sangue_coleta', 'resultado_sangue_coleta', 'diagnostico_diabetes', 'diagnostico_hipertensao', 'diagnostico_doenca_pulmao', 'diagnostico_doenca_cardiaca', 'diagnostico_depressao', 'diagnostico_cancer', 'afastado_trabalho', 'motivo_do_afastamento', 'remunerado_continua', 'mais_de_um_trabalho', 'tipo_trabalho', 'horas_trabalho_normal', 'tr

In [114]:
df.printSchema()

root
 |-- uf: string (nullable = true)
 |-- idade: integer (nullable = true)
 |-- sexo: string (nullable = true)
 |-- cor: string (nullable = true)
 |-- escolaridade: string (nullable = true)
 |-- ensino_publico_privado: string (nullable = true)
 |-- frequentou_escola: string (nullable = true)
 |-- tipo_moradia: string (nullable = true)
 |-- situacao_domiciliar: string (nullable = true)
 |-- rendimento: string (nullable = true)
 |-- restringiu_contato_fisico: string (nullable = true)
 |-- febre: string (nullable = true)
 |-- tosse: string (nullable = true)
 |-- dor_garganta: string (nullable = true)
 |-- dificuldade_respirar: string (nullable = true)
 |-- dor_cabeca: string (nullable = true)
 |-- dor_peito: string (nullable = true)
 |-- nausea: string (nullable = true)
 |-- nariz_entupido: string (nullable = true)
 |-- fadiga: string (nullable = true)
 |-- dor_olhos: string (nullable = true)
 |-- perda_olfato_paladar: string (nullable = true)
 |-- dor_muscular: string (nullable = true)

In [115]:
# resultado de testagem por região
df.createOrReplaceTempView('testagem_regiao')

testagem_regiao = spark.sql('''
    SELECT regiao, resultado_swab, resultado_sangue_dedo, resultado_sangue_coleta, count(resultado_sangue_coleta) as total
    FROM testagem_regiao
    WHERE resultado_swab != 'nao aplicavel' OR
        resultado_sangue_dedo != 'nao aplicavel' OR
        resultado_sangue_coleta != 'nao aplicavel'
    GROUP BY regiao, resultado_swab, resultado_sangue_dedo, resultado_sangue_coleta
    ORDER BY regiao, resultado_swab, resultado_sangue_dedo, resultado_sangue_coleta
''')

testagem_regiao.show()

+------------+--------------+---------------------+-----------------------+-----+
|      regiao|resultado_swab|resultado_sangue_dedo|resultado_sangue_coleta|total|
+------------+--------------+---------------------+-----------------------+-----+
|centro-oeste|  inconclusivo|         inconclusivo|          nao aplicavel|    3|
|centro-oeste|  inconclusivo|         inconclusivo|   nao recebeu o res...|    1|
|centro-oeste|  inconclusivo|        nao aplicavel|           inconclusivo|    2|
|centro-oeste|  inconclusivo|        nao aplicavel|          nao aplicavel|   15|
|centro-oeste|  inconclusivo|        nao aplicavel|               negativo|    4|
|centro-oeste|  inconclusivo|        nao aplicavel|               positivo|    2|
|centro-oeste|  inconclusivo|             negativo|          nao aplicavel|    2|
|centro-oeste|  inconclusivo|             negativo|               negativo|    1|
|centro-oeste| nao aplicavel|         inconclusivo|           inconclusivo|    1|
|centro-oeste| n

In [116]:
df.createOrReplaceTempView('sintomas')

# Calcula o total de registros com sintomas
total_registros_com_sintomas = spark.sql('''
    SELECT COUNT(*) as total
    FROM sintomas
    WHERE
        tosse != 'ignorado' OR
        dor_garganta != 'ignorado' OR
        dificuldade_respirar != 'ignorado' OR
        dor_cabeca != 'ignorado' OR
        dor_peito != 'ignorado' OR
        nausea != 'ignorado' OR
        fadiga != 'ignorado' OR
        dor_olhos != 'ignorado' OR
        perda_olfato_paladar != 'ignorado' OR
        dor_muscular != 'ignorado' OR
        diarreia != 'ignorado'
''').collect()[0]['total']

# Consulta para calcular o percentual de cada sintoma em relação ao total de registros com sintomas
percentuais = spark.sql(f'''
    SELECT
        SUM(CASE WHEN tosse = 'sim' THEN 1 ELSE 0 END) / {total_registros_com_sintomas} * 100 as percentual_tosse,
        SUM(CASE WHEN dor_garganta = 'sim' THEN 1 ELSE 0 END) / {total_registros_com_sintomas} * 100 as percentual_dor_garganta,
        SUM(CASE WHEN dificuldade_respirar = 'sim' THEN 1 ELSE 0 END) / {total_registros_com_sintomas} * 100 as percentual_dificuldade_respirar,
        SUM(CASE WHEN dor_cabeca = 'sim' THEN 1 ELSE 0 END) / {total_registros_com_sintomas} * 100 as percentual_dor_cabeca,
        SUM(CASE WHEN dor_peito = 'sim' THEN 1 ELSE 0 END) / {total_registros_com_sintomas} * 100 as percentual_dor_peito,
        SUM(CASE WHEN nausea = 'sim' THEN 1 ELSE 0 END) / {total_registros_com_sintomas} * 100 as percentual_nausea,
        SUM(CASE WHEN fadiga = 'sim' THEN 1 ELSE 0 END) / {total_registros_com_sintomas} * 100 as percentual_fadiga,
        SUM(CASE WHEN dor_olhos = 'sim' THEN 1 ELSE 0 END) / {total_registros_com_sintomas} * 100 as percentual_dor_olhos,
        SUM(CASE WHEN perda_olfato_paladar = 'sim' THEN 1 ELSE 0 END) / {total_registros_com_sintomas} * 100 as percentual_perda_olfato_paladar,
        SUM(CASE WHEN dor_muscular = 'sim' THEN 1 ELSE 0 END) / {total_registros_com_sintomas} * 100 as percentual_dor_muscular,
        SUM(CASE WHEN diarreia = 'sim' THEN 1 ELSE 0 END) / {total_registros_com_sintomas} * 100 as percentual_diarreia
    FROM sintomas
''').collect()[0]

# Exibe os percentuais
print("Percentual de sintomas positivos:")
print(f"Tosse: {percentuais['percentual_tosse']:.2f}%")
print(f"Dor de garganta: {percentuais['percentual_dor_garganta']:.2f}%")
print(f"Dificuldade respiratória: {percentuais['percentual_dificuldade_respirar']:.2f}%")
print(f"Dor de cabeça: {percentuais['percentual_dor_cabeca']:.2f}%")
print(f"Dor no peito: {percentuais['percentual_dor_peito']:.2f}%")
print(f"Náusea: {percentuais['percentual_nausea']:.2f}%")
print(f"Fadiga: {percentuais['percentual_fadiga']:.2f}%")
print(f"Dor nos olhos: {percentuais['percentual_dor_olhos']:.2f}%")
print(f"Perda de olfato ou paladar: {percentuais['percentual_perda_olfato_paladar']:.2f}%")
print(f"Dor muscular: {percentuais['percentual_dor_muscular']:.2f}%")
print(f"Diarreia: {percentuais['percentual_diarreia']:.2f}%")


Percentual de sintomas positivos:
Tosse: 1.34%
Dor de garganta: 1.10%
Dificuldade respiratória: 0.42%
Dor de cabeça: 1.65%
Dor no peito: 0.30%
Náusea: 0.35%
Fadiga: 0.72%
Dor nos olhos: 0.32%
Perda de olfato ou paladar: 0.39%
Dor muscular: 0.95%
Diarreia: 0.38%


In [117]:
df.createOrReplaceTempView('sintomas_sexo')

sintomas_sexo = spark.sql('''
    SELECT sexo, tosse, dor_garganta, dificuldade_respirar, dor_cabeca, dor_peito, nausea, fadiga, dor_olhos, perda_olfato_paladar, dor_muscular, diarreia,
        CASE WHEN tosse = 'sim' THEN 1 ELSE 0 END +
        CASE WHEN dor_garganta = 'sim' THEN 1 ELSE 0 END +
        CASE WHEN dificuldade_respirar = 'sim' THEN 1 ELSE 0 END +
        CASE WHEN dor_cabeca = 'sim' THEN 1 ELSE 0 END +
        CASE WHEN dor_peito = 'sim' THEN 1 ELSE 0 END +
        CASE WHEN nausea = 'sim' THEN 1 ELSE 0 END +
        CASE WHEN fadiga = 'sim' THEN 1 ELSE 0 END +
        CASE WHEN dor_olhos = 'sim' THEN 1 ELSE 0 END +
        CASE WHEN perda_olfato_paladar = 'sim' THEN 1 ELSE 0 END +
        CASE WHEN dor_muscular = 'sim' THEN 1 ELSE 0 END +
        CASE WHEN diarreia = 'sim' THEN 1 ELSE 0 END AS total_sintomas_positivo
    FROM sintomas_sexo
        WHERE
        (tosse = 'sim' OR
        dor_garganta = 'sim' OR
        dificuldade_respirar = 'sim' OR
        dor_cabeca = 'sim' OR
        dor_peito = 'sim' OR
        nausea = 'sim' OR
        fadiga = 'sim' OR
        dor_olhos = 'sim' OR
        perda_olfato_paladar = 'sim' OR
        dor_muscular = 'sim' OR
        diarreia = 'sim') AND
        (tosse != 'ignorado' AND
        dor_garganta != 'ignorado' AND
        dificuldade_respirar != 'ignorado' AND
        dor_cabeca != 'ignorado' AND
        dor_peito != 'ignorado' AND
        nausea != 'ignorado' AND
        fadiga != 'ignorado' AND
        dor_olhos != 'ignorado' AND
        perda_olfato_paladar != 'ignorado' AND
        dor_muscular != 'ignorado' AND
        diarreia != 'ignorado')
    GROUP BY sexo, tosse, dor_garganta, dificuldade_respirar, dor_cabeca, dor_peito, nausea, fadiga, dor_olhos, perda_olfato_paladar, dor_muscular, diarreia
    ORDER BY sexo, tosse, dor_garganta, dificuldade_respirar, dor_cabeca, dor_peito, nausea, fadiga, dor_olhos, perda_olfato_paladar, dor_muscular, diarreia
''')

sintomas_sexo.show()

+-----+-----+------------+--------------------+----------+---------+------+------+---------+--------------------+------------+--------+-----------------------+
| sexo|tosse|dor_garganta|dificuldade_respirar|dor_cabeca|dor_peito|nausea|fadiga|dor_olhos|perda_olfato_paladar|dor_muscular|diarreia|total_sintomas_positivo|
+-----+-----+------------+--------------------+----------+---------+------+------+---------+--------------------+------------+--------+-----------------------+
|homem| nao |        nao |                nao |      nao |     nao |  nao |  nao |     nao |                nao |        nao |     sim|                      1|
|homem| nao |        nao |                nao |      nao |     nao |  nao |  nao |     nao |                nao |         sim|    nao |                      1|
|homem| nao |        nao |                nao |      nao |     nao |  nao |  nao |     nao |                nao |         sim|     sim|                      2|
|homem| nao |        nao |              

In [118]:
# escolaridade por região
df.createOrReplaceTempView('escolaridade_regiao')

escolaridade_regiao = spark.sql('''
    SELECT regiao, escolaridade, count(*) as total_respostas
    FROM escolaridade_regiao
    GROUP BY regiao, escolaridade
    ORDER BY regiao, escolaridade
''')

escolaridade_regiao.show()

+------------+--------------------+---------------+
|      regiao|        escolaridade|total_respostas|
+------------+--------------------+---------------+
|centro-oeste|fundamental completa|           7540|
|centro-oeste|fundamental incom...|          39172|
|centro-oeste|      medio completo|          25668|
|centro-oeste|    medio incompleto|          12094|
|centro-oeste|pos mestrado dout...|           3995|
|centro-oeste|       sem instrucao|          12370|
|centro-oeste|   superior completo|          14045|
|centro-oeste| superior incompleto|           7589|
|    nordeste|fundamental completa|          18387|
|    nordeste|fundamental incom...|         133740|
|    nordeste|      medio completo|          72196|
|    nordeste|    medio incompleto|          34036|
|    nordeste|pos mestrado dout...|           6150|
|    nordeste|       sem instrucao|          49540|
|    nordeste|   superior completo|          22558|
|    nordeste| superior incompleto|          14292|
|       nort

In [119]:
# moradis por região
df.createOrReplaceTempView('moradia_regiao')

moradia_regiao = spark.sql('''
    SELECT regiao, tipo_moradia, count(*) as total_respostas
    FROM moradia_regiao
    GROUP BY regiao, tipo_moradia
    ORDER BY regiao, tipo_moradia
''')

moradia_regiao.show()

+------------+--------------------+---------------+
|      regiao|        tipo_moradia|total_respostas|
+------------+--------------------+---------------+
|centro-oeste|             alugado|          23454|
|centro-oeste|cedido de outra f...|           1600|
|centro-oeste|cedido por empreg...|           5802|
|centro-oeste|cedido por familiar |           9905|
|centro-oeste|      outra condicao|            370|
|centro-oeste|proprio_ainda pag...|          10448|
|centro-oeste|    proprio_ja pago |          70894|
|    nordeste|             alugado|          39577|
|    nordeste|cedido de outra f...|           4345|
|    nordeste|cedido por empreg...|           3686|
|    nordeste|cedido por familiar |          29082|
|    nordeste|      outra condicao|           1324|
|    nordeste|proprio_ainda pag...|          12012|
|    nordeste|    proprio_ja pago |         260873|
|       norte|             alugado|          16416|
|       norte|cedido de outra f...|           1230|
|       nort

In [120]:
df.createOrReplaceTempView('percentual_horas_trabalho')

percentual_horas_trabalho = spark.sql('''
    SELECT uf,
        ROUND((SUM(CASE WHEN horas_trabalho_normal > 0 THEN 1 ELSE 0 END) / COUNT(*)) * 100, 2) as percentual_horas_trabalho
    FROM percentual_horas_trabalho
    GROUP BY uf
    ORDER BY uf
''')

percentual_horas_trabalho.show()

+-------------------+-------------------------+
|                 uf|percentual_horas_trabalho|
+-------------------+-------------------------+
|               acre|                    32.75|
|            alagoas|                    26.42|
|              amapa|                    33.66|
|           amazonas|                    33.91|
|              bahia|                    32.23|
|              ceara|                    29.68|
|   distrito federal|                    41.27|
|     espirito santo|                    42.37|
|              goias|                    43.62|
|           maranhao|                    27.15|
|        mato grosso|                    43.86|
| mato grosso do sul|                    44.61|
|       minas gerais|                    40.82|
|               para|                    35.61|
|            paraiba|                    30.78|
|             parana|                    45.23|
|         pernambuco|                    32.34|
|              piaui|                   

In [121]:
df.createOrReplaceTempView('afastamento')

afastamento = spark.sql('''
    SELECT regiao, afastado_trabalho, motivo_do_afastamento
    FROM afastamento
      WHERE afastado_trabalho != 'nao aplicavel'
    GROUP BY regiao, afastado_trabalho,motivo_do_afastamento
    ORDER BY regiao, afastado_trabalho, motivo_do_afastamento
''')

afastamento.show()

+------------+-----------------+---------------------+
|      regiao|afastado_trabalho|motivo_do_afastamento|
+------------+-----------------+---------------------+
|centro-oeste|              nao|        nao aplicavel|
|centro-oeste|              sim| afastamento sem s...|
|centro-oeste|              sim|   fatores ocasionais|
|centro-oeste|              sim| ferias ou jornada...|
|centro-oeste|              sim| maternidade ou pa...|
|centro-oeste|              sim|         outro motivo|
|centro-oeste|              sim| outro tipo de lic...|
|centro-oeste|              sim| quarentena ou fer...|
|centro-oeste|              sim| saude ou acidente...|
|    nordeste|              nao|        nao aplicavel|
|    nordeste|              sim| afastamento sem s...|
|    nordeste|              sim|   fatores ocasionais|
|    nordeste|              sim| ferias ou jornada...|
|    nordeste|              sim| maternidade ou pa...|
|    nordeste|              sim|         outro motivo|
|    norde

In [122]:
df.createOrReplaceTempView('rendimento_regiao')

rendimento_regiao = spark.sql('''
    SELECT regiao, rendimento
    FROM rendimento_regiao
    GROUP BY regiao, rendimento
    ORDER BY regiao, rendimento
''')

rendimento_regiao.show()

+------------+----------------+
|      regiao|      rendimento|
+------------+----------------+
|centro-oeste|         0 a 100|
|centro-oeste|   1.601 a 3.000|
|centro-oeste| 10.001 a 50.000|
|centro-oeste|       101 a 300|
|centro-oeste|  3.001 a 10.000|
|centro-oeste|       301 a 600|
|centro-oeste|50.001 a 100.000|
|centro-oeste|       601 a 800|
|centro-oeste|     801 a 1.600|
|centro-oeste| mais de 100.000|
|centro-oeste|   nao aplicavel|
|    nordeste|         0 a 100|
|    nordeste|   1.601 a 3.000|
|    nordeste| 10.001 a 50.000|
|    nordeste|       101 a 300|
|    nordeste|  3.001 a 10.000|
|    nordeste|       301 a 600|
|    nordeste|50.001 a 100.000|
|    nordeste|       601 a 800|
|    nordeste|     801 a 1.600|
+------------+----------------+
only showing top 20 rows



In [123]:
df.createOrReplaceTempView('restringiu_contato_regiao')

restringiu_contato_regiao= spark.sql('''
    SELECT regiao, restringiu_contato_fisico, count(*) as total
    FROM restringiu_contato_fisico
    GROUP BY regiao, restringiu_contato_fisico
    ORDER BY regiao, restringiu_contato_fisico
''')

restringiu_contato_regiao.show()

+------------+-------------------------+------+
|      regiao|restringiu_contato_fisico| total|
+------------+-------------------------+------+
|centro-oeste|     ficou somente em ...| 15799|
|centro-oeste|            nao aplicavel|   759|
|centro-oeste|        reduziu o contato| 55568|
|centro-oeste|            sem restricao|  4272|
|centro-oeste|     so saiu para nece...| 46075|
|    nordeste|     ficou somente em ...| 53210|
|    nordeste|            nao aplicavel|  1193|
|    nordeste|        reduziu o contato|130390|
|    nordeste|            sem restricao| 16720|
|    nordeste|     so saiu para nece...|149386|
|       norte|     ficou somente em ...| 16718|
|       norte|            nao aplicavel|  1129|
|       norte|        reduziu o contato| 64342|
|       norte|            sem restricao|  8298|
|       norte|     so saiu para nece...| 51885|
|     sudeste|     ficou somente em ...| 41275|
|     sudeste|            nao aplicavel|  1250|
|     sudeste|        reduziu o contato|

In [124]:
df.createOrReplaceTempView('restringiu_contato_moradia')

restringiu_contato_moradia= spark.sql('''
    SELECT situacao_domiciliar, restringiu_contato_fisico, count(*) as total
    FROM restringiu_contato_fisico
    GROUP BY situacao_domiciliar, restringiu_contato_fisico
    ORDER BY situacao_domiciliar, restringiu_contato_fisico
''')

restringiu_contato_moradia.show()

+-------------------+-------------------------+------+
|situacao_domiciliar|restringiu_contato_fisico| total|
+-------------------+-------------------------+------+
|              rural|     ficou somente em ...| 42549|
|              rural|            nao aplicavel|   898|
|              rural|        reduziu o contato|100583|
|              rural|            sem restricao| 11106|
|              rural|     so saiu para nece...|118861|
|             urbana|     ficou somente em ...|112044|
|             urbana|            nao aplicavel|  4049|
|             urbana|        reduziu o contato|383930|
|             urbana|            sem restricao| 35088|
|             urbana|     so saiu para nece...|340089|
+-------------------+-------------------------+------+



In [125]:
df.createOrReplaceTempView('remunerado')

remunerado= spark.sql('''
    SELECT regiao, remunerado_continua, count(*) as total_remunerado
    FROM remunerado
    GROUP BY regiao, remunerado_continua
    ORDER BY regiao, remunerado_continua
''')

remunerado.show()

+------------+-------------------+----------------+
|      regiao|remunerado_continua|total_remunerado|
+------------+-------------------+----------------+
|centro-oeste|                nao|            1233|
|centro-oeste|      nao aplicavel|          120113|
|centro-oeste|     nao remunerado|              14|
|centro-oeste|                sim|            1113|
|    nordeste|                nao|            4339|
|    nordeste|      nao aplicavel|          343269|
|    nordeste|     nao remunerado|              92|
|    nordeste|                sim|            3199|
|       norte|                nao|            1576|
|       norte|      nao aplicavel|          138678|
|       norte|     nao remunerado|              35|
|       norte|                sim|            2083|
|     sudeste|                nao|            5134|
|     sudeste|      nao aplicavel|          327264|
|     sudeste|     nao remunerado|              53|
|     sudeste|                sim|            3582|
|         su

In [126]:
df.createOrReplaceTempView('tipo_moradia_remunerado')

tipo_moradia_remunerado= spark.sql('''
    SELECT tipo_moradia, remunerado_continua, count(*) as total_remunerado
    FROM remunerado
    GROUP BY tipo_moradia, remunerado_continua
    ORDER BY tipo_moradia, remunerado_continua
''')

tipo_moradia_remunerado.show()

+--------------------+-------------------+----------------+
|        tipo_moradia|remunerado_continua|total_remunerado|
+--------------------+-------------------+----------------+
|             alugado|                nao|            2089|
|             alugado|      nao aplicavel|          157530|
|             alugado|     nao remunerado|              12|
|             alugado|                sim|            1517|
|cedido de outra f...|                nao|             167|
|cedido de outra f...|      nao aplicavel|           12077|
|cedido de outra f...|     nao remunerado|               1|
|cedido de outra f...|                sim|              70|
|cedido por empreg...|                nao|             106|
|cedido por empreg...|      nao aplicavel|           25119|
|cedido por empreg...|                sim|              71|
|cedido por familiar |                nao|            1483|
|cedido por familiar |      nao aplicavel|           94463|
|cedido por familiar |     nao remunerad

In [127]:
df.createOrReplaceTempView('afastamento_remunerado')

afastamento_remunerado= spark.sql('''
    SELECT motivo_do_afastamento, afastado_trabalho, remunerado_continua, count(*) as total_remunerado
    FROM remunerado
    GROUP BY motivo_do_afastamento, afastado_trabalho, remunerado_continua
    ORDER BY motivo_do_afastamento, afastado_trabalho, remunerado_continua
''')

afastamento_remunerado.show()

+---------------------+-----------------+-------------------+----------------+
|motivo_do_afastamento|afastado_trabalho|remunerado_continua|total_remunerado|
+---------------------+-----------------+-------------------+----------------+
| afastamento sem s...|              sim|      nao aplicavel|            2476|
|   fatores ocasionais|              sim|      nao aplicavel|            1061|
| ferias ou jornada...|              sim|      nao aplicavel|            3561|
| maternidade ou pa...|              sim|      nao aplicavel|            1320|
|        nao aplicavel|              nao|      nao aplicavel|          482711|
|        nao aplicavel|    nao aplicavel|      nao aplicavel|          624881|
|         outro motivo|              sim|      nao aplicavel|            2226|
| outro tipo de lic...|              sim|      nao aplicavel|             428|
| quarentena ou fer...|              sim|                nao|           14084|
| quarentena ou fer...|              sim|     nao re